In [67]:
library(tidyverse)
library(nycflights13)
# this lecture uses some additional packages:
# install.packages(c("Lahman", "priceR", "DBI", "RSQLite", "priceR"))

# Lecture 09: More on relational data; SQL
<div style="border: 1px double black; padding: 10px; margin: 10px">

**After today's lecture you will:**
* Understand how to [filter data using joins](#Filtering-joins)
* Know to execute basic [SQL commands](#SQL-Commands)
</div>


## Review of last lecture
Last lecture we learned how to join data from one table to another. The main function we care about is `left_join(x, y, by=<key>)`, which tries to find a matching key in the `y` table for every row in the `x` table.

![match example](https://r4ds.hadley.nz/diagrams/join/left.png)

## Exercise
Let's refresh our memory about how joins work. Say I wanted to compare the distribution of departure delay when the weather was a) above or b) below freezing (32F/0C). 

In [54]:
# departure delay by freezing weather

## 🤔 Quiz
Assume every flight is full. The busiest travel day in terms of total # of passengers scheduled to depart is:

<ol style="list-style-type: upper-alpha;">
    <li> Jul. 3 </li>
    <li> Nov. 26</li>
    <li>Nov. 27</li>
    <li>Dec. 26</li>
    <li>Dec. 27</li>
</ol>

(Hint: the `planes` table tells you how many passengers each airplane holds.)

In [72]:
# total number of passengers

## Exercise
Another relational database we have seen in this class is the Lahman dataset on baseball stats:

In [73]:
library(Lahman)

The `Salaries` table contains information on player salaries, where they are available:

In [74]:
?Salaries

## Exercise
How does a player's salary compare with their career batting average (or some other stat)?

In [71]:
# salary vs batting average

## Filtering joins

Filtering joins allow us to filter rows on one table based on their presence or absence in another table. We've already seen some examples of this on the problem sets:

In [50]:
top5 = count(flights, carrier) %>% top_n(5) %>% print
top5$carrier
flights_top5 = filter(flights, carrier %in% top5$carrier) %>% print

Selecting by n


# A tibble: 5 x 2
  carrier     n
  <chr>   <int>
1 AA      32729
2 B6      54635
3 DL      48110
4 EV      54173
5 UA      58665


[1] "AA" "B6" "DL" "EV" "UA"

# A tibble: 248,312 x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      517            515         2      830            819
 2  2013     1     1      533            529         4      850            830
 3  2013     1     1      542            540         2      923            850
 4  2013     1     1      544            545        -1     1004           1022
 5  2013     1     1      554            600        -6      812            837
 6  2013     1     1      554            558        -4      740            728
 7  2013     1     1      555            600        -5      913            854
 8  2013     1     1      557            600        -3      709            723
 9  2013     1     1      557            600        -3      838            846
10  2013     1     1      558            600        -2      753            745
# … with 248,302 more rows,

What if we wanted to filter based on more than one variable? For example, based on busiest days of the year:

In [57]:
top6_days = count(flights, year, month, day) %>% top_n(6) %>% 
    arrange(n) %>% print

Selecting by n


# A tibble: 7 x 4
   year month   day     n
  <int> <int> <int> <int>
1  2013     7    18  1003
2  2013     7    25  1003
3  2013     7     8  1004
4  2013     7    10  1004
5  2013    12     2  1004
6  2013     7    11  1006
7  2013    11    27  1014


In [1]:
# manual filter join

### Semi-join
`semi_join(x, y)` keeps all the observations in `x` that are also in `y`.
![semi join](http://r4ds.had.co.nz/diagrams/join-semi.png)

`semi_join()` takes the same `key=` options as the other join commands.

In [56]:
# All flights in 6 busiest days of year:
semi_join(flights, top6_days, key = c("year", "month", "day")) %>% print

Joining, by = c("year", "month", "day")


# A tibble: 7,038 x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013    11    27       28           1930       298      129           2049
 2  2013    11    27      505            500         5      703            651
 3  2013    11    27      514            515        -1      743            808
 4  2013    11    27      535            540        -5      907            850
 5  2013    11    27      540            545        -5      815            835
 6  2013    11    27      551            600        -9      920            854
 7  2013    11    27      551            600        -9      918            849
 8  2013    11    27      554            600        -6      719            719
 9  2013    11    27      554            600        -6      707            717
10  2013    11    27      554            600        -6      907            846
# … with 7,028 more rows, and

### Exercise
Of the 336,776 flights in `flights`, what % of them departed on one of the 20 busiest days of the year?

In [13]:
# flights departing in the 20 top busiest days

### Anti-join
The opposite of a semi-join is an anti-join:
![anti-join](http://r4ds.had.co.nz/diagrams/join-anti.png)

## Filtering joins

Filtering joins allow us to filter rows on one table based on their presence or absence in another table. We've already seen some examples of this on the problem sets:

In [61]:
dest_top6 <- count(flights, dest) %>% top_n(6)
filter(flights, dest %in% dest_top6$dest) %>% nrow

Selecting by n


[1] 94326

### Semi-join
`semi_join(x, y)` keeps all the observations in `x` that are also in `y`.
![semi join](http://r4ds.had.co.nz/diagrams/join-semi.png)

In [74]:
# find all the flight with destinations in the top 6

## 🤔 Quiz
The top ten most popular days to fly were:

```count(flights, year, month, day) %>% top_n(10)```

What is the overall fraction of flights that left on these ten days?

<ol style="list-style-type: upper-alpha;">
    <li>0.024</li>
    <li>0.042</li>
    <li>1.000</li>
    <li>0.001</li>
    <li>0.039</li>
</ol>

### Anti-join
`anti_join(x, y)` does the opposite of `semi_join`: it *drops* all the observations in `x` that are also in `y`.
![anti join](https://r4ds.hadley.nz/diagrams/join/anti.png)

Anti-joins are useful for finding mismatches in joins. Last lecture we saw that some 50k entries in `flights` have `tailnum`s that do not exist in the `planes` table. To find out more about these flights we could use an anti-join to select only those rows:

In [10]:
missing_planes = anti_join(flights, planes, by="tailnum") %>% 
                 filter(!is.na(tailnum)) %>% print

# A tibble: 50,094 x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      558            600        -2      753            745
 2  2013     1     1      559            600        -1      941            910
 3  2013     1     1      600            600         0      837            825
 4  2013     1     1      602            605        -3      821            805
 5  2013     1     1      608            600         8      807            735
 6  2013     1     1      611            600        11      945            931
 7  2013     1     1      623            610        13      920            915
 8  2013     1     1      624            630        -6      840            830
 9  2013     1     1      628            630        -2     1137           1140
10  2013     1     1      629            630        -1      824            810
# … with 50,084 more rows, a

Do we see any pattern in this table that could explain the missing tail numbers?

In [11]:
missing_planes %>% sample_n(100) %>% print(width = Inf)

# A tibble: 100 x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013    12     7     1551           1515        36     1723           1705
 2  2013     2    16      915            920        -5     1207           1245
 3  2013     4    29      858            900        -2     1143           1203
 4  2013     8    27     1932           1930         2     2238           2250
 5  2013    11    13     1504           1510        -6     2019           2020
 6  2013     8    24      943            950        -7     1201           1205
 7  2013     9    10     1421           1425        -4     1629           1650
 8  2013     5    31      733            740        -7     1025           1040
 9  2013     5    19     1846           1800        46     1959           1950
10  2013     1    23     1041           1050        -9     1229           1250
   arr_delay carrier flight tai

Interestingly, the top 5 most frequent `tailnum`s in the overall data are almost the same:

In [12]:
flights %>% drop_na(tailnum) %>% left_join(planes, by = "tailnum") %>% 
    count(tailnum, model) %>% top_n(5)

Selecting by n


  tailnum model  n  
1 N711MQ  G1159B 486
2 N713MQ  NA     483
3 N722MQ  NA     513
4 N723MQ  NA     507
5 N725MQ  NA     575

In [13]:
planes %>% filter(tailnum == "N711MQ")

  tailnum year type                    manufacturer         model  engines
1 N711MQ  1976 Fixed wing multi engine GULFSTREAM AEROSPACE G1159B 2      
  seats speed engine   
1 22    NA    Turbo-jet

The one plane that is represented is N711M. You can [look it up](https://flightaware.com/resources/registration/N711MQ) in an online database and find out it's a private jet. My guess is the others are private jets too.

<td>How many planes flew to destinations which are <i>not</i> found in the <code>airports</code> table?</td>

# SQL

SQL stands for "Structured Query Language". Many large databases are stored in SQL format, and you will probably encounter one if you work on big data and/or at a large company. 

In [8]:
# SQLite database
library(nycflights13)
library(DBI)
library(RSQLite)
con <- dbConnect(SQLite(), ":memory:")
copy_to(con, flights, "flights")
copy_to(con, airports, "airports")
copy_to(con, airlines, "airlines")
copy_to(con, weather, "weather")
q <- function(...) dbGetQuery(con, ...) %>% as_tibble

### Selecting data from a table
The SQL syntax for selecting column(s) from a table is
```{sql}
SELECT <col1>, <col2>, ..., <coln> FROM <table>
```
Note the similarity to the corresponding `tidyverse` command:
```{r}
select(<table>, <col1>, <col2>, ..., <coln>)
```

In [10]:
# select one column

The special keyword `*` means "select everything" and is equivalent to `dplyr`'s `everything()`:

In [35]:
# select all columns

If you have a really big table, SQL allows you to `LIMIT` the number of rows it returns.

In [36]:
# query first ten rows of flights

### Distinct rows
Writing `SELECT DISTINCT` instead of `SELECT` will give every unique row in the data set:

In [48]:
# distinct year, month, day in flights

### Filtering

The SQL syntax for filtering rows in a table uses the `WHERE` clause:
```{sql}
SELECT * FROM <table> WHERE dest = "IAH"
```
This is the same as:
```{r}
filter(<table>, dest == "IAH")
```
Note that SQL uses a single `=` to check equality!

In [37]:
# select where

### Inequality in SQL
The write that something is *not equal* in SQL, we traditionally use the `<>` operator:

In [53]:
# all flights that are not destined for LAX

Newer dialects of SQL (including SQLite) seem to let you use `!=` as well though.

### Missing data
In SQL, missing data is coded as `NULL`. This is a special value which is analogous to `NA` in R. 

In [39]:
# select non-null dep time

### Summarizing

The SQL syntax for summarizing is using the `GROUP BY` clause:
```{sql}
SELECT AVG(<col>) AS avg_col FROM <table> GROUP BY(<group cols>)
```
This is the same as:
```{r}
<table> %>% group_by(<group cols>) %>% summarize(avg_col = mean(<col>))
```

In [40]:
# select average distance and count for each flight

You can find a list of the aggregation functions that SQLite supports [here](https://www.sqlite.org/lang_aggfunc.html). This is a somewhat limited set compared to richer SQL engines like MySQL.

### Joins

The SQL syntax for joins:
```{sql}
SELECT * FROM <table> LEFT JOIN <other_table> ON <left_key_col> = <right_key_col>
```
This is the same as:
```{r}
left_join(<table>, <other_table>, by = c("<left_key_col>" = "<right_key_col>"))
```

In [41]:
# join planes by tailnum

Note here that SQL requires us to be explicit about which columns we are `SELECT`ing when joining multiple tables. Each column name must be prefixed with the name of the table in which it resides.

### Ranking

Ranking operates slightly differently in SQL than dplyr. The overall syntax is:


    SELECT RANK() OVER (
	    PARTITION BY <columns>
	    ORDER BY <expression1> [ASC|DESC], ...
    ) RankedCol FROM <table>
    
    
The easiest way to learn it is to see some examples.

In [20]:
# rank based on departure delay

## Subqueries

An aspect of SQL I find confusing is that you can filter (`WHERE ...`) based on a ranking condition:

In [ ]:
# filtering a rank, won't work

Instead, you can write a *subquery*, basically stitching two SQL calls together:

    WITH (<a SQL query>) AS table
    SELECT <stuff> FROM table WHERE <conditions>
    

In [44]:
# filtering with rank

## Advanced joins in SQL
SQL is more general in specifying the join condition. Whereas in tidyverse it must be a key, in
SQL it can be a general logical condition.

## Example
What is the closest airport to `DTW`?

In [42]:
# find the nearest neighbor to each airport

Let's use this to build a map of every airport's nearest neighbor:

In [45]:
# compute air_nn, every airport's nearest neighbor



In [43]:
air_nn  %>% filter(lat < 50, lat > 23) %>%
    ggplot + 
    geom_segment(aes(x = lon, y = lat, xend = lon2, yend = lat2), color="blue") +
    geom_point(aes(x = lon, y = lat)) + 
    coord_quickmap()

ERROR: Error in filter(., lat < 50, lat > 23): object 'air_nn' not found


## Exercise
In 1985 a hurricane made landfall on Long Island, NY between JFK Airport and Islip.
What was the name of that hurricane?

In [156]:
jfk <- airports %>% filter(faa == "JFK")
jfk$lon
jfk$lat
storms %>% mutate(dist = sqrt((lat -+ jfk$lat)^2 + (long - jfk$lon)^2)) %>% select(dist, everything()) %>%
    arrange(dist) %>% slice(1:3)

[1] -73.77893

[1] 40.63975

  dist      name   year month day hour lat  long  status              category
1 0.2817433 Floyd  1999 9     17   0   40.6 -73.5 tropical storm      0       
2 0.4805718 Gloria 1985 9     27  16   40.6 -73.3 hurricane           1       
3 0.5881268 Chris  1988 8     30   0   41.2 -73.6 tropical depression -1      
  wind pressure ts_diameter hu_diameter
1 50    980     NA          NA         
2 75    961     NA          NA         
3 20   1008     NA          NA         